# Applied Machine Learning homework3

Huibo Zhao hz2480
Lingjie Xu lx2222

# Perform General Data Preprocessing

In [33]:
import os
import pandas as pd
import math
import numpy as np
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV, Lasso, Ridge
from sklearn.model_selection import GridSearchCV

In [2]:
# Extract those files that contain our training data
path = os.getcwd()
files = os.listdir(path)
xlsx_files = [f for f in files if f[-4:] == 'xlsx' and f[0] != '~' and f[0:4] != '2018']

# Putting all the contents into dataframe for viewing and accessing
df = pd.DataFrame()
for f in xlsx_files:
    data = pd.read_excel(f, 'FEguide')
    df = df.append(data)
    
train_row_count = df.shape[0] # record the number of training data

# appending data of 2018
xlsx_test = [f for f in files if f[-4:] == 'xlsx' and f[0] != '~' and f[0:4] == '2018']
test_data = pd.read_excel(xlsx_test[0], 'FEguide')
df = df.append(test_data)

In [3]:
# We extract the target value
y = df['Comb Unrd Adj FE - Conventional Fuel'].as_matrix()

# Examine if the target contains any empty or nan value
# If it does, we need to drop the row without a valid target value
# Fortunately, we don't find such row
for i in y:
    if isinstance(i,float) and math.isnan(i):
        print(i)

In [4]:
dropout_features = list()


# Professor had named a few features that we cannot use, we extract the name of those features
da = np.array(df.columns)
drop_index = list()
for i in range(len(da)):
    if "FE" in da[i]:
        drop_index.append(da[i])
    elif "MPG" in da[i]:
        drop_index.append(da[i])
    elif "CO2" in da[i]:
        drop_index.append(da[i])
    elif "Smog" in da[i]:
        drop_index.append(da[i])
    elif "Guzzler" in da[i]:
        drop_index.append(da[i])
    elif "EPA" in da[i]:
        drop_index.append(da[i])
    elif "GHG" in da[i]:
        drop_index.append(da[i])

dropout_features.extend(drop_index)
df_1 = df.drop(drop_index, axis=1) ##########first drop : drop disallowed features##########
############################################################################################

# We examine each feature and count how many nan values it have
# Save results in a dictionary

column_names = df_1.columns.values
nan_dict = {}

for column_name in column_names:
    count = 0
    for i in range(0,df_1.shape[0]):
        if isinstance(df_1.iloc[i][column_name],float) and math.isnan(df_1.iloc[i][column_name]):
            count += 1
    nan_dict[column_name] = count
    
drop_2 = list()
for key in nan_dict:
    if nan_dict[key]>=2000:
        drop_2.append(key)

dropout_features.extend(drop_2)
df_2 = df_1.drop(drop_2, axis=1) ##########second drop : drop features without enough valid entry#########
##########################################################################################################

In [5]:
# After the second drop, we left with 48 features
# We found 8 features have at least 1 nan values  
# We have to handle that 
count_dict = {}
column_names = df_2.columns.values
for column_name in column_names:
    count = 0
    for i in range(0,df_2.shape[0]):
        if isinstance(df_2.iloc[i][column_name],float) and math.isnan(df_2.iloc[i][column_name]):
            count += 1
    count_dict[column_name] = count
    
        
for key in count_dict:
    if count_dict[key] > 0:
        print(str(key) + ": " + str(count_dict[key]))


Max Ethanol % - Gasoline: 107
Descriptor - Model Type (40 Char or less): 1391
Car/Truck Category - Cash for Clunkers Bill.: 1690
Unique Label?: 420
Label Recalc?: 248
Var Valve Timing Desc: 151
$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label) : 1301


In [6]:
dropout_features.append("Car/Truck Category - Cash for Clunkers Bill.")
df_3 = df_2.drop("Car/Truck Category - Cash for Clunkers Bill.", axis=1) #### third drop ####
####### Car/Truck features is useless after examining it ####################################

dropout_features.append("Release Date") ###### Fourth drop ##################################
df_4 = df_3.drop("Release Date", axis=1) ##### We temporarily drop the date here ############


dropout_features.append("Descriptor - Model Type (40 Char or less)")  ######## fifth drop ###
df_5 = df_4.drop("Descriptor - Model Type (40 Char or less)", axis=1) #######################

#################### We have done all dropping features by this point #######################
#################### All dropout features are stored in dropout_features ####################


In [7]:
################################ We fill out those four features manually ###################
index_ = df_5.columns
adjust_index = list()
adjust_index.append(list(index_).index("Max Ethanol % - Gasoline"))
adjust_index.append(list(index_).index("Unique Label?"))
adjust_index.append(list(index_).index("Label Recalc?"))
adjust_index.append(list(index_).index("Var Valve Timing Desc"))
adjust_index

for i in adjust_index:
    for j in range(df_5.shape[0]):
        if j != df_5.shape[0] - 1:
            if isinstance(df_5.iat[j, i], float) and math.isnan(df_5.iat[j, i]):
                if isinstance(df_5.iat[j + 1, i], float) and math.isnan(df_5.iat[j + 1, i]):
                    df_5.iat[j, i] = df_5.iat[j - 1, i]
                else:
                    m = random.randint(0,1)
                    if m == 0:
                        df_5.iat[j, i] = df_5.iat[j - 1, i]
                    else:
                        df_5.iat[j, i] = df_5.iat[j + 1, i]
        else:
            if isinstance(df_5.iat[j, i],float) and math.isnan(df_5.iat[j, i]):
                df_5.iat[j, i] = df_5.iat[j - 1, i]
##############################################################################################

############################# using get_dummies to fill out discrete features ################
df_6 = pd.get_dummies(df_5)
##############################################################################################

In [8]:
import fancyimpute

############################# using fancy_impute to impute data entries ######################
X = df_6.as_matrix()
mice = fancyimpute.MICE(verbose=0)
X_fancy_mice = mice.complete(X)
#############################################################################################

# We assume the data are distributed i.i.d but we know we are going to use
# 2018 data as test data
X_15_17 = X_fancy_mice[0:train_row_count, :]
X_18 = X_fancy_mice[train_row_count: , :]
y_15_17 = y[0:train_row_count,]
y_18 = y[train_row_count:,]

Using TensorFlow backend.


In [9]:
# Evaluate
# ridge_pipe = make_pipeline(StandardScaler(), RidgeCV())
# scores = cross_val_score(ridge_pipe, X_fancy_mice, y, cv=10)
# np.mean(scores), np.std(scores)

In [10]:
lasso_pipe = make_pipeline(StandardScaler(), LassoCV())
lasso_pipe.fit(X_15_17, y_15_17)
lasso_pipe.score(X_18, y_18)
# scores = cross_val_score(lasso_pipe, X_fancy_mice, y, cv=10)
# np.mean(scores), np.std(scores)

0.82832590331041489

In [34]:
ridge_pipe_s = make_pipeline(StandardScaler(), Ridge())

In [35]:
ridge_pipe_s.get_params().keys()

dict_keys(['memory', 'steps', 'standardscaler', 'ridge', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'ridge__alpha', 'ridge__copy_X', 'ridge__fit_intercept', 'ridge__max_iter', 'ridge__normalize', 'ridge__random_state', 'ridge__solver', 'ridge__tol'])

In [37]:
X_train, X_val, y_train, y_val = train_test_split(X_15_17, y_15_17, random_state=0)

In [39]:
param_grid = {'ridge__alpha': np.logspace(-3, 3, 7)}
print(param_grid)
grid = GridSearchCV(ridge_pipe_s, param_grid, cv=10)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.score(X_val, y_val))

{'ridge__alpha': array([  1.00000000e-03,   1.00000000e-02,   1.00000000e-01,
         1.00000000e+00,   1.00000000e+01,   1.00000000e+02,
         1.00000000e+03])}
{'ridge__alpha': 10.0}
0.940590204594


In [40]:
pd.DataFrame(grid.cv_results_).columns

Index(['mean_fit_time', 'mean_score_time', 'mean_test_score',
       'mean_train_score', 'param_ridge__alpha', 'params', 'rank_test_score',
       'split0_test_score', 'split0_train_score', 'split1_test_score',
       'split1_train_score', 'split2_test_score', 'split2_train_score',
       'split3_test_score', 'split3_train_score', 'split4_test_score',
       'split4_train_score', 'split5_test_score', 'split5_train_score',
       'split6_test_score', 'split6_train_score', 'split7_test_score',
       'split7_train_score', 'split8_test_score', 'split8_train_score',
       'split9_test_score', 'split9_train_score', 'std_fit_time',
       'std_score_time', 'std_test_score', 'std_train_score'],
      dtype='object')

In [42]:
ridge_pipe_s_t = make_pipeline(StandardScaler(), Ridge(alpha=10.0))

In [44]:
ridge_pipe_s_t.fit(X_15_17, y_15_17)
ridge_pipe_s_t.score(X_18, y_18)

0.88746749333588415